# Semantic Annotation of Data using JSON Linked Data

## Abstract

The Earthcube Geosemantics Framework (http://ecgs.ncsa.illinois.edu/) developed a prototype of a decentralized framework that combines the Linked Data and RESTful web services to annotate, connect, integrate, and reason about integration of geoscience resources. The framework allows the semantic enrichment of web resources and semantic mediation among heterogeneous geoscience resources, such as models and data. 

This notebook provides examples on how the Semantic Annotation Service can be used to manage linked controlled vocabularies using JSON Linked Data (JSON-LD), including how to query the built-in RDF graphs for existing linked standard vocabularies based on the Community Surface Dynamics Modeling System (CSDMS), Observations Data Model (ODM2) and Unidata udunits2 vocabularies, how to query build-in crosswalks between CSDMS and ODM2 vocabularies using SKOS, and how to add new linked vocabularies to the service. JSON-LD based definitions provided by these endpoints will be used to annotate sample data available within the IML Critical Zone Observatory data repository using the Clowder Web Service API (https://data.imlczo.org/). By supporting JSON-LD, the Semantic Annotation Service and the Clowder framework provide examples on how portable and semantically defined metadata can be used to better annotate data across repositories and services.

## Introduction

### Geosemantics Framework

We face many challenges in the process of extracting meaningful information from data [add examples of challenges related to integration of models and data]. Frequently, these obstacle  compel scientists to perform the integration of models with data manually. Manual integration becomes exponentially difficult when a user aims to integrate long-tail data (data collected by individual researchers or small research groups) and long-tail models (models developed by individuals or small modeling communities). We focus on these long-tail resources because despite their often-narrow scope, they have significant impacts in scientific studies and present an opportunity for addressing critical gaps through automated integration. The goal of the Goesemantics Framework is to provide a framework rooted in semantic techniques and approaches to support “long-tail” models and data integration.

### Clowder Data Management Framework


### Linked Data

The Linked Data paradigm emerged in the context of Semantic Web technologies for publishing and sharing data over the Web. It connects related individual Web resources in a Graph database, where resources represent the graph nodes, and an edge connects a pair of nodes. Publishing and linking scientific resources using Semantic Web technologies require that the user community follows the three principles of Linked Data:

1.  Each resource needs to be represented using a unique Uniform Resource Identifier (URI), which consists of: (i) A Uniform Resource Locator (URL) to define the server path over the Web, and (ii) A Uniform Resource Name (URN) to describe the exact name of the resource.

2. The relationships between resources are described using the triple format, where a subject S has a predicate P with an object O. A predicate is either an undirected relationship (bi-directional), where it connects two entities in both ways or a directed relationship (uni-directional), where the presence of a relationship between two entities in one direction does not imply the presence of a reverse relationship. The triple format is the structure unit for the Linked Data system. 

3. The HyperText Transfer Protocol (HTTP) is used as a universal access mechanism for resources on the Web. 

For more information about linked data, please visit https://www.w3.org/standards/semanticweb/data.

## Basic Requirements

We first setup some basic requirements used through out the notebook. 

In [1]:
import requests
import json
import ipywidgets as widgets

from IPython.display import display

gsis_host = 'http://hcgs.ncsa.illinois.edu'

We will be using two main services. The first is the Geosemantics Integration Service (GSIS) available at `http://hcgs.ncsa.illinois.edu`. This service provides support for standandard vocabularies and methods for transforming typical strings used for tracking time, space and physical variables into well formed Linked Data documents. Because all the endpoints we will be using are public, no credentials are required and we just need to set the URL of the host.

## Temporal Annotation Services
Time values are represented in UTC (Coordinated Universal Time) format. Times are expressed in local time, together with a time zone offset in hours and minutes. For more information, please visit https://www.w3.org/TR/NOTE-datetime for more information.

### Time Instant Annotation
Query parameters: 
* **time** (string): time value in UTC format

In [2]:
# Get a temporal annotation for a time instant in a JSON-LD format.
time = '2014-01-01T08:01:01-09:00'
r = requests.get(f"{gsis_host}/gsis/sas/temporal?time={time}")
r.json()

{'@context': {'dc': 'http://purl.org/dc/elements/1.1/',
  'dcterms': 'http://purl.org/dc/terms/',
  'xsd': 'http://www.w3.org/2001/XMLSchema#',
  'time': 'http://www.w3.org/2006/time#',
  'tzont': 'http://www.w3.org/2006/timezone-us'},
 '@id': 'http://ecgs.ncsa.illinois.edu/time_instant',
 '@type': 'time:Instant',
 'dc:date': "yyyy-MM-dd'T'HH:mm:ssZ",
 'time:DateTimeDescription': {'time:year': '2014',
  'time:month': '1',
  'time:day': '1',
  'tzont': '-09:00',
  'time:hours': '8',
  'time:minutes': '1',
  'time:seconds': '1'}}

### Time Interval Annotation
Query parameters: 
* **beginning** (string): time value in UTC format.
* **end** (string): time value in UTC format.

In [3]:
# Get a temporal annotation for a time interval in a JSON-LD format.
beginning = '2014-01-01T08:01:01-10:00'
end = '2014-12-31T08:01:01-10:00'
r = requests.get(f"{gsis_host}/gsis/sas/temporal?beginning={beginning}&end={end}")
r.json()

{'@context': {'dc': 'http://purl.org/dc/elements/1.1/',
  'dcterms': 'http://purl.org/dc/terms/',
  'xsd': 'http://www.w3.org/2001/XMLSchema#',
  'time': 'http://www.w3.org/2006/time#',
  'tzont': 'http://www.w3.org/2006/timezone-us'},
 '@id': 'http://ecgs.ncsa.illinois.edu/time_interval',
 '@type': 'time:Interval',
 'time:Duration': {'time:hasBeginning': {'dc:date': "yyyy-MM-dd'T'HH:mm:ssZ",
   'time:DateTimeDescription': {'time:year': 2014,
    'time:month': 1,
    'time:day': 1,
    'tzont': '-10:00',
    'time:hours': 8,
    'time:minutes': 1,
    'time:seconds': 1}},
  'time:hasEnd': {'dc:date': "yyyy-MM-dd'T'HH:mm:ssZ",
   'time:DateTimeDescription': {'time:year': 2014,
    'time:month': 12,
    'time:day': 31,
    'tzont': '-10:00',
    'time:hours': 8,
    'time:minutes': 1,
    'time:seconds': 1}}}}

### TIme Series Annotation
Query parameters:
* **beginning** (string): time value in UTC format.
* **end** (string): time value in UTC format.
* **interval** (float): time step.

In [4]:
# Get a temporal annotation for a time series in a JSON-LD format.
beginning = '2014-01-01T08:01:01-10:00'
end = '2014-03-01T08:01:01-10:00'
interval = '4'
r = requests.get(f"{gsis_host}/gsis/sas/temporal?beginning={beginning}&end={end}&interval={interval}")
r.json()

{'@context': {'dc': 'http://purl.org/dc/elements/1.1/',
  'dcterms': 'http://purl.org/dc/terms/',
  'xsd': 'http://www.w3.org/2001/XMLSchema#',
  'time': 'http://www.w3.org/2006/time#',
  'tzont': 'http://www.w3.org/2006/timezone-us'},
 '@id': 'http://ecgs.ncsa.illinois.edu/time_series',
 'time:Duration': {'time:hasBeginning': {'dc:date': "yyyy-MM-dd'T'HH:mm:ssZ",
   'time:DateTimeDescription': {'time:year': 2014,
    'time:month': 1,
    'time:day': 1,
    'tzont': '-10:00',
    'time:hours': 8,
    'time:minutes': 1,
    'time:seconds': 1}},
  'time:hasEnd': {'dc:date': "yyyy-MM-dd'T'HH:mm:ssZ",
   'time:DateTimeDescription': {'time:year': 2014,
    'time:month': 3,
    'time:day': 1,
    'tzont': '-10:00',
    'time:hours': 8,
    'time:minutes': 1,
    'time:seconds': 1}},
  'time:temporalUnit': {'@type': 'time:unitSecond', '@value': 4}}}

## Semantic Annotation Service

### Lists the names of all graphs in the Knowledge base

Returns a list of all the graphs stored in the knowledge base.

In [5]:
r = requests.get(gsis_host + "/gsis/listGraphNames")
r.json()

{'graph_names': ['csdms',
  'odm2-vars',
  'udunits2-base',
  'udunits2-derived',
  'udunits2-accepted',
  'udunits2-prefix',
  'google-unit',
  'model-2',
  'model-3',
  'data-1',
  'data-2',
  'data-3',
  'variable_name_crosswalk',
  'variable_name_crosswalk-owl',
  'variable_name_crosswalk-skos',
  'model_test',
  'model_test11',
  'config_vars.ttl',
  'model-x',
  'Info',
  'Inf',
  'demo-model',
  'csv-mappings',
  'models_graph9d7d400f53864989a05d3ae539f30a78',
  'models_graph37baec3114d74ca6abd72cce75f966db',
  'models_graphe604316f14334985aaf4ebd6fe220e77',
  'models_graph26e29f5026664f11b244072bf6956f74']}

In [6]:
r = requests.get(gsis_host + "/gsis/read?graph=udunits2-prefix")
r.json()

{'@graph': [{'@id': 'http://mmisw.org/ont/mmitest/udunits2-prefix/Prefix',
   '@type': 'owl:Class',
   'rdfs:label': 'Prefix',
   'subClassOf': 'skos:Concept'},
  {'@id': 'http://mmisw.org/ont/mmitest/udunits2-prefix/atto',
   '@type': 'http://mmisw.org/ont/mmitest/udunits2-prefix/Prefix',
   'http://mmisw.org/ont/mmitest/udunits2-prefix/name': 'atto',
   'http://mmisw.org/ont/mmitest/udunits2-prefix/symbol': 'a',
   'http://mmisw.org/ont/mmitest/udunits2-prefix/value': '1e-18'},
  {'@id': 'http://mmisw.org/ont/mmitest/udunits2-prefix/centi',
   '@type': 'http://mmisw.org/ont/mmitest/udunits2-prefix/Prefix',
   'http://mmisw.org/ont/mmitest/udunits2-prefix/name': 'centi',
   'http://mmisw.org/ont/mmitest/udunits2-prefix/symbol': 'c',
   'http://mmisw.org/ont/mmitest/udunits2-prefix/value': '.01'},
  {'@id': 'http://mmisw.org/ont/mmitest/udunits2-prefix/deci',
   '@type': 'http://mmisw.org/ont/mmitest/udunits2-prefix/Prefix',
   'http://mmisw.org/ont/mmitest/udunits2-prefix/name': 'deci

In [7]:
r = requests.get(gsis_host + "/gsis/sas/vars/list?term=wind+speed")
r.json()

['csn:earth_surface_wind__range_of_speed',
 'csn:land_surface_wind__reference_height_speed',
 'csn:land_surface_wind__speed_reference_height',
 'csn:projectile_origin_wind__speed',
 'odm2:windGustSpeed',
 'odm2:windSpeed']

In [8]:
r = requests.get(gsis_host + "/gsis/CSNqueryName?graph=csdms&name=air__dynamic_shear_viscosity")
r.json()

{'name': 'air__dynamic_shear_viscosity',
 'type': 'http://ecgs.ncsa.illinois.edu/2015/csn/name',
 'object_fullname': 'air',
 'quantity_fullname': 'dynamic_shear_viscosity',
 'base_object': 'air',
 'base_quantity': 'viscosity',
 'object_part': ['air'],
 'quantity_part': ['dynamic', 'shear', 'viscosity']}

In [9]:
r = requests.get(gsis_host + "/gsis/CSNqueryName?graph=odm2-vars&name=windSpeed")
r.json()

{}

## Variable Annotation Services

### Lists the names of all graphs in the Knowledge base

In [10]:
# Get a lists of the names of all graphs in the Knowledge base
r = requests.get(f"{gsis_host}/gsis/listGraphNames")
r.json()

{'graph_names': ['csdms',
  'odm2-vars',
  'udunits2-base',
  'udunits2-derived',
  'udunits2-accepted',
  'udunits2-prefix',
  'google-unit',
  'model-2',
  'model-3',
  'data-1',
  'data-2',
  'data-3',
  'variable_name_crosswalk',
  'variable_name_crosswalk-owl',
  'variable_name_crosswalk-skos',
  'model_test',
  'model_test11',
  'config_vars.ttl',
  'model-x',
  'Info',
  'Inf',
  'demo-model',
  'csv-mappings',
  'models_graph9d7d400f53864989a05d3ae539f30a78',
  'models_graph37baec3114d74ca6abd72cce75f966db',
  'models_graphe604316f14334985aaf4ebd6fe220e77',
  'models_graph26e29f5026664f11b244072bf6956f74']}

### List the content of a Graph (for example, CSDMS Standard Names)

In [11]:
# Get the content stored in a specific graph in a JSON-LD format.
graph = 'csdms'
r = requests.get(f"{gsis_host}/gsis/read?graph={graph}")
r.json().get('@graph')[0:10] # We just show the top 10 results, to see all results remove the slice operator [0:10]

[{'@id': 'csn:air__dielectric_constant',
  '@type': 'csn:name',
  'csn:base_object': 'air',
  'csn:base_quantity': 'constant',
  'csn:object_fullname': 'air',
  'csn:object_part': 'air',
  'csn:quantity_fullname': 'dielectric_constant',
  'csn:quantity_part': ['dielectric', 'constant']},
 {'@id': 'csn:air__dynamic_shear_viscosity',
  '@type': 'csn:name',
  'csn:base_object': 'air',
  'csn:base_quantity': 'viscosity',
  'csn:object_fullname': 'air',
  'csn:object_part': 'air',
  'csn:quantity_fullname': 'dynamic_shear_viscosity',
  'csn:quantity_part': ['dynamic', 'shear', 'viscosity']},
 {'@id': 'csn:air__dynamic_volume_viscosity',
  '@type': 'csn:name',
  'csn:base_object': 'air',
  'csn:base_quantity': 'viscosity',
  'csn:object_fullname': 'air',
  'csn:object_part': 'air',
  'csn:quantity_fullname': 'dynamic_volume_viscosity',
  'csn:quantity_part': ['dynamic', 'viscosity', 'volume']},
 {'@id': 'csn:air__kinematic_shear_viscosity',
  '@type': 'csn:name',
  'csn:base_object': 'air',


### List of CSDMS Standard Names and ODM2 Variable Names

In [12]:
# Get the CSDMS Standard Names as a flat list.
r = requests.get(f"{gsis_host}/gsis/sas/sn/csn")
csn_terms = r.json()
csn_terms[0:20] # We just show the top 20 results, to see all results remove the slice operator [0:20]

['air__dielectric_constant',
 'air__dynamic_shear_viscosity',
 'air__dynamic_volume_viscosity',
 'air__kinematic_shear_viscosity',
 'air__kinematic_volume_viscosity',
 'air__volume-specific_isochoric_heat_capacity',
 'air_helium-plume__richardson_number',
 'air_radiation~visible__speed',
 'air_water~vapor__dew_point_temperature',
 'air_water~vapor__saturated_partial_pressure',
 'aircraft__flight_duration',
 'airfoil__drag_coefficient',
 'airfoil__lift_coefficient',
 'airfoil_curve~enclosing__circulation',
 'airplane__altitude',
 'airplane__mach_number',
 'airplane_wing__span',
 'air~dry__mass-specific_gas_constant',
 'air~dry_water~vapor__gas_constant_ratio',
 'aluminum__mass-specific_isobaric_heat_capacity']

In [13]:
[i for i in csn_terms if 'partial_pressure' in i]

['air_water~vapor__saturated_partial_pressure',
 'atmosphere_air_carbon-dioxide__equilibrium_partial_pressure',
 'atmosphere_air_carbon-dioxide__partial_pressure',
 'atmosphere_air_carbon-dioxide__saturated_partial_pressure',
 'atmosphere_air_water~vapor__equilibrium_partial_pressure',
 'atmosphere_air_water~vapor__partial_pressure',
 'atmosphere_air_water~vapor__saturated_partial_pressure',
 'atmosphere_bottom_air_carbon-dioxide__equilibrium_partial_pressure',
 'atmosphere_bottom_air_carbon-dioxide__partial_pressure',
 'atmosphere_bottom_air_carbon-dioxide__saturated_partial_pressure',
 'atmosphere_bottom_air_water~vapor__equilibrium_partial_pressure',
 'atmosphere_bottom_air_water~vapor__partial_pressure',
 'atmosphere_bottom_air_water~vapor__saturated_partial_pressure',
 'atmosphere_carbon-dioxide__partial_pressure',
 'atmosphere_water~vapor__partial_pressure',
 'atmosphere_water~vapor__saturated_partial_pressure',
 'sea_surface_air_carbon-dioxide__partial_pressure',
 'sea_surface_a

In [14]:
# Get the ODM2 Variable Names as a flat list.
r = requests.get(f"{gsis_host}/gsis/sas/sn/odm2")
r.json()[0:20] # We just show the top 20 results, to see all results remove the slice operator [0:20]

['19_Hexanoyloxyfucoxanthin',
 '1_1_1_Trichloroethane',
 '1_1_2_2_Tetrachloroethane',
 '1_1_2_Trichloroethane',
 '1_1_Dichloroethane',
 '1_1_Dichloroethene',
 '1_2_3_Trimethylbenzene',
 '1_2_4_5_Tetrachlorobenzene',
 '1_2_4_Trichlorobenzene',
 '1_2_4_Trimethylbenzene',
 '1_2_Dibromo_3_Chloropropane',
 '1_2_Dichlorobenzene',
 '1_2_Dichloroethane',
 '1_2_Dichloropropane',
 '1_2_Dimethylnaphthalene',
 '1_2_Dinitrobenzene',
 '1_2_Diphenylhydrazine',
 '1_3_5_Trimethylbenzene',
 '1_3_Dichlorobenzene',
 '1_3_Dimethyladamantane']

### Search Across Registered Graphs

In [15]:
# Get all properties of a given CSDMS Standard Name from a specific graph in a JSON-LD format.
graph = 'csdms'
name = 'air__dynamic_shear_viscosity'
r = requests.get(f"{gsis_host}/gsis/CSNqueryName?graph={graph}&name={name}")
r.json()

{'name': 'air__dynamic_shear_viscosity',
 'type': 'http://ecgs.ncsa.illinois.edu/2015/csn/name',
 'object_fullname': 'air',
 'quantity_fullname': 'dynamic_shear_viscosity',
 'base_object': 'air',
 'base_quantity': 'viscosity',
 'object_part': ['air'],
 'quantity_part': ['dynamic', 'shear', 'viscosity']}

### Units

In [16]:
# Get the list of udunits2 units in JSON format.
r = requests.get(f"{gsis_host}/gsis/sas/unit/udunits2")
r.json()[0:20] # We just show the top 20 results, to see all results remove the slice operator [0:20]

['ampere',
 'arc_degree',
 'arc_minute',
 'arc_second',
 'candela',
 'coulomb',
 'day',
 'degree_Celsius',
 'electronvolt',
 'farad',
 'gram',
 'gray',
 'henry',
 'hertz',
 'hour',
 'joule',
 'katal',
 'kelvin',
 'kilogram',
 'liter']

In [17]:
## Get the list of Google units in JSON format.
r = requests.get(gsis_host + "/gsis/sas/unit/google")
r.json()[0:20] # We just show the top 20 results, to see all results remove the slice operator [0:20]

['acre',
 'acre-foot',
 'Algerian dinar',
 'ampere',
 'ampere hour',
 'amu',
 'arc minute',
 'arc second',
 'are',
 'Argentine peso',
 'Astronomical Unit',
 'ATA pica',
 'ATA point',
 'atmosphere',
 'atomic mass unit',
 'Australian cent',
 'Australian dollar',
 'Bahrain dinar',
 "baker's dozen",
 'bar']

## Clowder

The second services is a specific Clowder instance developed by the NSF Intensively Managed Landscape Critical Zone Observatory. We will be retrieving data from it and uploading data and metadata back to it using simple HTTP based web services.

Because we will be adding information to the Clowder instance, we will be required to register an account on the IMLCZO Clowder instance and create an API key and added to the cell below.

In [18]:
clowder_host = 'https://data.imlczo.org/clowder'

# Please create an API key as described above
# If using a .env file is confusing you can manually set the key
# clowder_key = 'copy and paste your key here'
%load_ext dotenv
%dotenv
import os
clowder_key = os.getenv("CLOWDER-KEY")

headers = {'Content-type': 'application/json', 'X-API-Key': clowder_key}

## Search by metadata

Search by keyword

In [19]:
query = 'test'
url = "{}/api/search?query={}".format(clowder_host, query)
r = requests.get(url, headers=headers)
r.raise_for_status()
r.json()

{'count': 8,
 'size': 8,
 'scanned_size': 240,
 'results': [{'id': '56d79957e4b0b55c0889bea2',
   'name': 'Test dataset',
   'description': 'Testing permissions',
   'created': 'Wed Mar 02 19:54:31 CST 2016',
   'thumbnail': '56d79975e4b0b55c0889bec7',
   'authorId': '53adb361bb0d14bfccb04617',
   'spaces': ['56d7987ee4b0b55c0889bea0'],
   'resource_type': 'dataset'},
  {'id': '5ebb1c2d4f0c0ce4611383bb',
   'collectionname': 'Tests',
   'description': 'tests',
   'created': 'Tue May 12 16:59:09 CDT 2020',
   'thumbnail': None,
   'authorId': '53adb361bb0d14bfccb04617',
   'resource_type': 'collection'},
  {'id': '56d79976e4b0b55c0889bed1',
   'name': 'carbon-emissions.pdf',
   'status': 'PROCESSED',
   'thumbnail': '56d79977e4b0b55c0889bee8',
   'created': 'Wed Mar 02 19:55:02 CST 2016',
   'resource_type': 'file'},
  {'id': '5979f72a4f0c2e3a97a0948a',
   'name': 'dinoskull.obj',
   'status': 'PROCESSED',
   'thumbnail': None,
   'created': 'Thu Jul 27 09:22:34 CDT 2017',
   'resource_

Search by `ODM2 Variable Name = precipitation`

In [27]:
query = '"ODM2 Variable Name":"precipitation"'
url = "{}/api/search?query={}".format(clowder_host, query)
r = requests.get(url, headers=headers)
search = r.json()
datasetId = search.get('results')[0].get('id')
print('Dataset id: ' + datasetId)

Dataset id: 596faa3b4f0c0b1c81fa42de


In [28]:
query = '"ODM2 Variable Name":"precipitation"'
url = "{}/api/search?query={}".format(clowder_host, query)
r = requests.get(url, headers=headers)
search = r.json()
datasetId = search.get('results')[0].get('id')
print('Dataset id: ' + datasetId)

Dataset id: 596faa3b4f0c0b1c81fa42de


In [29]:
# List files in dataset
url = "{}/api/datasets/{}/files".format(clowder_host, datasetId)
r = requests.get(url)
files = r.json()
# Download the first file
fileId = files[0].get('id')
fileName = files[0].get('filename')
url = "{}/api/files/{}/blob".format(clowder_host, fileId)
r = requests.get(url)
open(fileName, 'wb').write(r.content)

2211973

### Create new dataset

In [32]:
def create_dataset(name, description, access, space, collection):
    '''
     params: name, description, access: PUBLIC vs PRIVATE, 
         space: a list of string can be empty,
         collection: a list of string, can be empty
    '''
    url = "{}/api/datasets/createempty".format(clowder_host)
    payload = json.dumps({'name':name, 
                          'description':description,
                          'access':access,
                          'space':space,
                          'collection':collection}) 

    r = requests.post(url,
                     data=payload,
                     headers=headers)
    print(r.status_code)
    print(r.text)
    return r.json()

In [33]:
new_dataset = create_dataset(name="new dataset", description="...", access="PRIVATE", 
               space=['5ebb1c114f0c0ce46113839d'],
              collection=['5ebb1c2d4f0c0ce4611383bb'])
new_dataset_id = new_dataset.get('id')
print(f'View at {clowder_host}/datasets/{new_dataset_id}')

AttributeError: 'list' object has no attribute 'dumps'

## Upload file to new dataset

In [ ]:
url = "{}/api/uploadToDataset/{}".format(clowder_host, new_dataset_id)
files = {'file': open(fileName, 'rb')}
r = requests.post(url, files=files, headers={'X-API-Key': clowder_key})
r.raise_for_status()
print(r.json())
fileId = r.json().get('id')
print(f'View at {clowder_host}/files/{fileId}')

## Add metadata to new file

In [ ]:
import json
url = "{}/api/files/{}/metadata.jsonld".format(clowder_host, fileId)
payload = {
    "@context":[
        "https://clowder.ncsa.illinois.edu/contexts/metadata.jsonld"
    ],
    "agent":{
        "@type":"cat:extractor",
        "name":"ECGS Notebook",
        "extractor_id":"https://clowder.ncsa.illinois.edu/api/extractors/ecgs"
    },
    "content":{
        "foo": "bar"
    }
}
r = requests.post(url, headers = headers, data=json.dumps(payload))
r.json()

In [ ]:
# The context file describes the basic elements of a Clowder metadata document
r = requests.get('https://clowder.ncsa.illinois.edu/contexts/metadata.jsonld')
r.json()